In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter, OrderedDict

from in_out import load_data, transform_y_to_only_one_class

Using TensorFlow backend.


In [2]:
DATA_PATH = '../definitive_data_folder'

LABELS =['ECTODERM', 'NEURAL_CREST', 'MESODERM', 'ENDODERM']

In [3]:
X_train, Y_train, X_test, Y_test = load_data('dataset')
X_train.head()

Loaded


,donor_age_at_diagnosis,connected_components,connected_components_max_size,DEL_16,DUP_16,TRA_16,h2hINV_16,t2tINV_16,DEL_20,DUP_20,...,proportion_chr_7,proportion_chr_8,proportion_chr_9,proportion_chr_X,proportion_DEL_Y,proportion_DUP_Y,proportion_TRA_Y,proportion_h2hINV_Y,proportion_t2tINV_Y,proportion_chr_Y
sampleID,,,,,,,,,,,,,,,,,,,,,
f8a165b0-51df-413c-b7c6-c7fc80547db4,71,2.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.045455,0.000000,0.068182,0.090909,0.0,0.0,0.000000,0.0,0.0,0.000000
17a66d24-de77-4f2a-a8ad-d2c6a69e7671,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
e2b5c926-491d-430d-8d9c-5c73d7752d1b,39,3.0,15.0,3.0,4.0,0.0,2.0,1.0,0.0,2.0,...,0.102190,0.036496,0.029197,0.000000,0.0,0.0,0.007299,0.0,0.0,0.007299
f87eb1b5-1712-ca1f-e040-11ac0c483848,57,3.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.082192,0.246575,0.369863,0.027397,0.0,0.0,0.000000,0.0,0.0,0.000000
82b8cda8-fbff-455e-b0db-7ff6528bd6c8,74,3.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.072464,0.000000,0.000000,0.057971,0.0,0.0,0.000000,0.0,0.0,0.000000


# Hyper parameter tuning

In [11]:
def calculate_best_hyperparameters(X_train, Y_train, n_iter_search):
    param_dist = {"max_depth": stats.randint(2, 20),
                      "min_samples_split": stats.randint(2, 11),
                      "min_samples_leaf": stats.randint(1, 20),
                      "bootstrap": [True, False],
                      "max_features": ['auto', 'log2', None],
                      "criterion": ["gini", "entropy"]}

    clf = RandomForestClassifier(n_estimators=100, class_weight='balanced')

    random_search = RandomizedSearchCV(clf, param_distributions=param_dist, iid=False,
                                       n_iter=n_iter_search, pre_dispatch=3, n_jobs=-1)

    random_search.fit(X_train, Y_train.values.ravel())
    best_params = random_search.best_params_
    best_params['n_estimators'] = 100
    best_params['class_weight'] = 'balanced'
    print('The best hyperparameters are: ', best_params)
    return best_params

best_params = calculate_best_hyperparameters(X_train,Y_train, n_iter_search=500)
best_params

('The best hyperparameters are: ', {'bootstrap': True, 'min_samples_leaf': 3, 'min_samples_split': 6, 'criterion': 'entropy', 'max_features': None, 'max_depth': 19})


{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 19,
 'max_features': None,
 'min_samples_leaf': 3,
 'min_samples_split': 6}

The best hyperparameters: 
{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 19,
 'max_features': None,
 'min_samples_leaf': 3,
 'min_samples_split': 6, 
 'class_weight': 'balanced',
 'n_estimators': 100}

In [4]:
best_hyperparameters = {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 19, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 6, 'class_weight': 'balanced', 'n_estimators': 100} 

# Feature ranking


In [24]:
def feature_extractor(X_train,Y_train, best_hyperparameters, number_of_iterations):
    features = list(X_train.columns)
    feature_importance = {feature:0 for feature in features}
    for i in range(number_of_iterations):
        if i%10==0:
            print i
        random_forest = RandomForestClassifier(**best_hyperparameters)
        random_forest = random_forest.fit(X_train, Y_train)
        local_fi = np.array(pd.DataFrame(random_forest.feature_importances_,
                                       index=X_train.columns,
                                       columns=['importance']).sort_values('importance', ascending=False).index)
        j = 1
        for feature in local_fi:
            feature_importance[feature] += j
            j +=1
    D = OrderedDict((k, v) for k, v in sorted(feature_importance.iteritems(), key=lambda kv: kv[1]))
    with open(DATA_PATH + '/feature_ranking.json', 'w') as f:
        f.write(json.dumps(D))
    return D

ordered_features = feature_extractor(X_train,Y_train, best_hyperparameters, number_of_iterations = 500)
ordered_features

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490


OrderedDict([('donor_age_at_diagnosis', 500),
             ('female', 1000),
             ('tumor_stage1_Primary_tumour', 1500),
             ('tumor_stage2_solid_tissue', 2009),
             ('DEL', 2496),
             ('tumor_stage2_other', 3600),
             ('chr_8', 3739),
             ('TRA', 3750),
             ('number_of_breaks', 3957),
             ('proportion_DUP', 5259),
             ('proportion_DEL', 5455),
             ('tumor_stage2_lymph_node', 6224),
             ('proportion_chr_9', 6364),
             ('t2tINV', 7568),
             ('proportion_DEL_14', 8092),
             ('proportion_chr_5', 8417),
             ('proportion_h2hINV_19', 8443),
             ('proportion_chr_1', 8599),
             ('proportion_TRA', 9689),
             ('proportion_chr_8', 10171),
             ('proportion_t2tINV', 10520),
             ('proportion_chr_12', 11987),
             ('proportion_h2hINV', 12119),
             ('proportion_chr_3', 12132),
             ('proportion_chr_6'

In [6]:
def load_feature_ranking():
    with open(DATA_PATH + '/feature_ranking.json', 'r') as read_file:
        feature_ranking = json.loads(read_file.read())
    D = OrderedDict((k, v) for k, v in sorted(feature_ranking.iteritems(), key=lambda kv: kv[1]))
    return D
ordered_features = load_feature_ranking()

In [7]:
ordered_features.keys()

[u'donor_age_at_diagnosis',
 u'female',
 u'tumor_stage1_Primary_tumour',
 u'tumor_stage2_solid_tissue',
 u'DEL',
 u'tumor_stage2_other',
 u'chr_8',
 u'TRA',
 u'number_of_breaks',
 u'proportion_DUP',
 u'proportion_DEL',
 u'tumor_stage2_lymph_node',
 u'proportion_chr_9',
 u't2tINV',
 u'proportion_DEL_14',
 u'proportion_chr_5',
 u'proportion_h2hINV_19',
 u'proportion_chr_1',
 u'proportion_TRA',
 u'proportion_chr_8',
 u'proportion_t2tINV',
 u'proportion_chr_12',
 u'proportion_h2hINV',
 u'proportion_chr_3',
 u'proportion_chr_6',
 u'proportion_chr_14',
 u'proportion_chr_4',
 u'proportion_chr_2',
 u'proportion_DEL_4',
 u'proportion_chr_7',
 u'chr_3',
 u'proportion_chr_11',
 u'proportion_chr_17',
 u'proportion_DEL_5',
 u'TRA_8',
 u'proportion_DUP_12',
 u'proportion_chr_10',
 u'proportion_DEL_9',
 u'DUP',
 u'proportion_DEL_3',
 u'proportion_chr_19',
 u'proportion_DEL_7',
 u'proportion_TRA_8',
 u'proportion_TRA_7',
 u'proportion_chr_X',
 u'proportion_TRA_9',
 u'proportion_TRA_5',
 u'chr_1',
 u'p

In [8]:
def best_n_features(n):
    ordered_features = load_feature_ranking()
    return ordered_features.keys()[:n]

best_n_features(5)

[u'donor_age_at_diagnosis',
 u'female',
 u'tumor_stage1_Primary_tumour',
 u'tumor_stage2_solid_tissue',
 u'DEL']

# Feature ranking One vs All

In [ ]:
def feature_extractor_one_vs_all(X_train,Y_train, best_hyperparameters, number_of_iterations):
    all_rankings = {}
    for class_name in LABELS: 
        Y_train_class = transform_y_to_only_one_class(Y_train, class_name)
        features = list(X_train.columns)
        feature_importance = {feature:0 for feature in features}
        for i in range(number_of_iterations):
            if i%10==0:
                print i
            random_forest = RandomForestClassifier(**best_hyperparameters)
            random_forest = random_forest.fit(X_train, Y_train_class)
            local_fi = np.array(pd.DataFrame(random_forest.feature_importances_,
                                           index=X_train.columns,
                                           columns=['importance']).sort_values('importance', ascending=False).index)
            j = 1
            for feature in local_fi:
                feature_importance[feature] += j
                j +=1
        D = OrderedDict((k, v) for k, v in sorted(feature_importance.iteritems(), key=lambda kv: kv[1]))
        all_rankings[class_name] = D
        with open(DATA_PATH + '/feature_ranking_'+ class_name +'.json', 'w') as f:
            f.write(json.dumps(D))
    return all_rankings

all_rankings = feature_extractor_one_vs_all(X_train,Y_train, best_hyperparameters, number_of_iterations=2)
all_rankings

0
